In [1]:
#import all the modeules we've used before

import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
no_nontoxicnans_xtest = pd.read_csv('no_nontoxic_nans_x_test.csv', index_col=0)

In [3]:

no_nontoxicnans_xtrain = pd.read_csv('no_nontoxic_nans_x_train.csv', index_col=0)

no_nontoxicnans_xval = pd.read_csv('no_nontoxic_nans_x_val.csv', index_col=0)
no_nontoxicnans_ytest = pd.read_csv('no_nontoxic_nans_y_test.csv', index_col=0)
no_nontoxicnans_ytrain = pd.read_csv('no_nontoxic_nans_y_train.csv', index_col= 0)
no_nontoxicnans_yval = pd.read_csv('no_nontoxic_nans_y_val.csv', index_col=0)

In [4]:
no_nontoxicnans_ytrain

,toxic
2839,0
4142,0
5263,1
4453,0
818,0
...,...
1303,0
3982,1
2812,0
3836,1


In [15]:
no_nontoxicnans_xtrain.isna().stack()[lambda x: x].index
no_nontoxicnans_xtrain.index[no_nontoxicnans_xtrain.isna().any(axis=1)]

rows, cols = np.where(no_nontoxicnans_xtrain.isna())

for r, c in zip(rows, cols):
    print(f"NaN at row={no_nontoxicnans_xtrain.index[r]}, column={no_nontoxicnans_xtrain.columns[c]}")



NaN at row=4513, column=MaxAbsEStateIndex
NaN at row=4513, column=MaxEStateIndex
NaN at row=4513, column=MinAbsEStateIndex
NaN at row=4513, column=MinEStateIndex
NaN at row=4513, column=qed
NaN at row=4513, column=SPS
NaN at row=4513, column=MolWt
NaN at row=4513, column=HeavyAtomMolWt
NaN at row=4513, column=ExactMolWt
NaN at row=4513, column=NumValenceElectrons
NaN at row=4513, column=NumRadicalElectrons
NaN at row=4513, column=MaxPartialCharge
NaN at row=4513, column=MinPartialCharge
NaN at row=4513, column=MaxAbsPartialCharge
NaN at row=4513, column=MinAbsPartialCharge
NaN at row=4513, column=FpDensityMorgan1
NaN at row=4513, column=FpDensityMorgan2
NaN at row=4513, column=FpDensityMorgan3
NaN at row=4513, column=BCUT2D_MWHI
NaN at row=4513, column=BCUT2D_MWLOW
NaN at row=4513, column=BCUT2D_CHGHI
NaN at row=4513, column=BCUT2D_CHGLO
NaN at row=4513, column=BCUT2D_LOGPHI
NaN at row=4513, column=BCUT2D_LOGPLOW
NaN at row=4513, column=BCUT2D_MRHI
NaN at row=4513, column=BCUT2D_MRLOW


In [5]:
num_rows_with_nan = no_nontoxicnans_xtrain.isna().any(axis=1).sum()
print(num_rows_with_nan)

84


In [24]:
mask = no_nontoxicnans_xtrain.notna().all(axis=1)

no_nontoxicnans_xtrain_clean = no_nontoxicnans_xtrain[mask]
no_nontoxicnans_ytrain_clean = no_nontoxicnans_ytrain[mask]

mask2 = no_nontoxicnans_xtest.notna().all(axis=1)

no_nontoxicnans_xtest_clean = no_nontoxicnans_xtest[mask2]
no_nontoxicnans_ytest_clean = no_nontoxicnans_ytest[mask2]

mask3 = no_nontoxicnans_xval.notna().all(axis=1)

no_nontoxicnans_xval_clean = no_nontoxicnans_xval[mask3]
no_nontoxicnans_yval_clean = no_nontoxicnans_yval[mask3]

In [25]:
pca_vis = PCA(n_components=2)

### fit and transform the data with the pca (note .fit_transform can be used here) - transform is somewhat similar to predict

pca_vis.fit(no_nontoxicnans_xtrain_clean)
pca_2 = pca_vis.transform(no_nontoxicnans_xtrain_clean)

### I have then converted this to a DataFrame

df_pca_vis = pd.DataFrame(pca_2, columns=['PC1', 'PC2'])

In [26]:
df_pca_vis

,PC1,PC2
0,1.894389,-2.836298
1,-1.317584,-4.127984
2,-7.189565,2.767101
3,-6.654386,-0.498383
4,-6.977640,1.241773
...,...,...
3602,4.579751,-0.589831
3603,-3.454608,-2.886404
3604,-3.482373,2.672191
3605,5.588645,-2.608962


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn import metrics
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [35]:
# if y is a Series
y_binary_labelstrain = no_nontoxicnans_ytrain_clean.iloc[:, 0].map({0: "non-toxic", 1: "toxic"})
y_binary_labelstest = no_nontoxicnans_ytest_clean.iloc[:, 0].map({0: "non-toxic", 1: "toxic"})

In [36]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),   # keep 95% variance
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

# train
pipeline.fit(no_nontoxicnans_xtrain_clean, y_binary_labelstrain)

# predict
y_pred = pipeline.predict(no_nontoxicnans_xtest_clean)

from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_binary_labelstest, y_pred))

Accuracy: 0.6508344030808729


In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

# Pipeline WITH PCA
pipeline_pca = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.9)),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

scores_pca = cross_val_score(pipeline_pca, no_nontoxicnans_xtrain_clean, no_nontoxicnans_ytrain_clean, cv=5, scoring="accuracy")
print("With PCA:", scores_pca.mean())


c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when

With PCA: 0.7357874758434154


c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [38]:
pipeline_no_pca = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

scores_no_pca = cross_val_score(pipeline_no_pca, no_nontoxicnans_xtrain_clean, no_nontoxicnans_ytrain_clean, cv=5, scoring="accuracy")
print("Without PCA:", scores_no_pca.mean())


Without PCA: 0.7335737145623384


c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when

In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "pca__n_components": [5, 10, 20, 0.9, 0.95],
    "knn__n_neighbors": [3, 5, 7]
}

grid = GridSearchCV(pipeline_pca, param_grid, cv=5, scoring="accuracy")
grid.fit(no_nontoxicnans_xtrain_clean, no_nontoxicnans_ytrain_clean)
print("Best params:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)


c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when

Best params: {'knn__n_neighbors': 7, 'pca__n_components': 0.95}
Best CV accuracy: 0.7391154175679363


c:\Users\rf25545\.conda\envs\chem-ml-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [47]:
pipeline2 = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),   # keep 95% variance
    ("knn", KNeighborsClassifier(n_neighbors=7))
])

# train
pipeline2.fit(no_nontoxicnans_xtrain_clean, y_binary_labelstrain)

# predict
y_pred = pipeline2.predict(no_nontoxicnans_xtest_clean)

from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_binary_labelstest, y_pred))

####### OPTIMAL K NEAREST NEIGHBOURS MODEL - PCA ANALYSIS PRIOR TO K NEAREST NEIGHBOURS MAKES NO DIFFERENCE - CHECKED WITH CV
####### 7 NEIGHBOURS IS THE OPTIMUM

Accuracy: 0.6521181001283697
